In [1]:
!python train.py \
  --dataroot /home/liujia/g_linux/Phantom_Carotid/ \
  --name 61_Texture_Upsample_Fix \
  --checkpoints_dir /mnt/g/train_data/checkpoints \
  --dir_lq Recon_LQ_03 \
  --dir_sq Recon_SQ_75 \
  --model augan \
  --netG res_unet_3d \
  --input_nc 1 \
  --output_nc 1 \
  --batch_size 2 \
  --patch_size_d 128 \
  --patch_size_h 64 \
  --patch_size_w 64 \
  --norm batch \
  --gpu_ids 0 \
  --n_epochs 100 \
  --n_epochs_decay 100 \
  --lr 0.0002 \
  --norm_min -60.0 \
  --norm_max 0.0 \
  --lambda_pixel 100.0 \
  --lambda_gan 1.0 \
  --lambda_ffl 1.0 \
  --lambda_tv 0.1 \
  --lambda_bg 0.0 \
  --epoch_count 1

----------------- Options ---------------
                attn_temp: 1.0                           
               batch_size: 2                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: /mnt/g/train_data/checkpoints 	[default: ./checkpoints]
           continue_train: False                         
                 cropSize: 256                           
                 dataroot: /home/liujia/g_linux/Phantom_Carotid/	[default: None]
             dataset_mode: unaligned                     
                   dir_lq: Recon_LQ_03                   
                   dir_sq: Recon_SQ_75                   
                direction: AtoB                          
              display_dir: ./checkpoints                 
             display_freq: 400                           
                    epoch: latest                        
              epoch_count: 1                             
                 gan_mode: 

In [ ]:
!python test.py \
  --dataroot /home/liujia/g_linux/test/simu_1channel \
--name 61_Texture_Upsample_Fix \
  --checkpoints_dir /mnt/g/train_data/checkpoints \
  --results_dir /mnt/g/train_data/results \
  --dir_lq Recon_LQ_03 \
  --dir_sq Recon_SQ_75 \
  --model augan \
  --netG res_unet_3d \
  --input_nc 1 \
  --output_nc 1 \
  --norm batch \
  --gpu_ids 0 \
  --norm_min -60.0 \
  --norm_max 0.0 \
  --epoch latest \
  --num_test 300

In [3]:
import os
import numpy as np
import nibabel as nib
from tqdm import tqdm

# ================= 配置区域 =================
DATA_ROOT = "/home/liujia/g_linux/Phantom_Carotid/"
DIR_LQ = "Recon_LQ_03"
DIR_SQ = "Recon_SQ_75"
# ===========================================

def check_nifti_data_v3():
    lq_path = os.path.join(DATA_ROOT, DIR_LQ)
    sq_path = os.path.join(DATA_ROOT, DIR_SQ)

    if not os.path.exists(lq_path) or not os.path.exists(sq_path):
        print(f"❌ 错误：找不到文件夹！")
        return

    # 获取所有 .nii 文件
    lq_files = sorted([f for f in os.listdir(lq_path) if f.endswith('.nii')])
    sq_files = sorted([f for f in os.listdir(sq_path) if f.endswith('.nii')])

    print(f"📂 数据集路径: {DATA_ROOT}")
    print(f"   LQ 文件数: {len(lq_files)}")
    print(f"   SQ 文件数: {len(sq_files)}")
    
    # === 智能配对逻辑 (忽略 _lq 和 _sq 后缀) ===
    # 建立映射表: 核心名 -> 完整文件名
    # 例如: 'Carotid_061' -> 'Carotid_061_lq.nii'
    lq_map = {f.replace('_lq.nii', '').replace('.nii', ''): f for f in lq_files}
    sq_map = {f.replace('_sq.nii', '').replace('.nii', ''): f for f in sq_files}
    
    lq_keys = set(lq_map.keys())
    sq_keys = set(sq_map.keys())
    
    common_keys = sorted(list(lq_keys & sq_keys))
    missing_in_sq = lq_keys - sq_keys
    missing_in_lq = sq_keys - lq_keys

    print("-" * 60)
    if len(missing_in_sq) > 0:
        print(f"❌ 严重警告：有 {len(missing_in_sq)} 个文件在 LQ 有，但 SQ 缺失！会导致训练错位！")
        print(f"   示例缺失核心名: {list(missing_in_sq)[:3]}")
    if len(missing_in_lq) > 0:
        print(f"❌ 严重警告：有 {len(missing_in_lq)} 个文件在 SQ 有，但 LQ 缺失！")

    if len(common_keys) == 0:
        print("❌ 错误：完全没找到配对！请检查文件名是否包含 _lq / _sq 以外的差异。")
        return

    print(f"✅ 智能配对成功: {len(common_keys)} 对数据")
    print("-" * 60)
    print("🚀 开始核心检查：数值范围 & NaN...")

    cnt_suspicious = 0
    cnt_nan = 0
    
    # 遍历配对好的文件
    for key in tqdm(common_keys):
        fname_l = lq_map[key]
        fname_s = sq_map[key]
        
        path_l = os.path.join(lq_path, fname_l)
        path_s = os.path.join(sq_path, fname_s)

        try:
            # 加载
            data_l = nib.load(path_l).get_fdata().astype(np.float32)
            data_s = nib.load(path_s).get_fdata().astype(np.float32)

            # 1. 检查形状
            if data_l.shape != data_s.shape:
                print(f"❌ [形状不对] {key}: LQ={data_l.shape}, SQ={data_s.shape}")
                continue

            # 2. 检查 NaN
            if np.isnan(data_l).any() or np.isnan(data_s).any():
                print(f"❌ [含 NaN] {key}")
                cnt_nan += 1
                continue

            # 3. 检查数值范围 (核心中的核心)
            max_l, min_l = data_l.max(), data_l.min()
            max_s, min_s = data_s.max(), data_s.min()
            
            # 报警条件：如果最大值 > 10 (说明可能是 0-255)，或者最大值 < -100 (说明数据全是背景)
            # 正常的 dB 数据，最大值应该在 0 附近 (比如 -5 到 5)，最小值在 -60 到 -100
            if max_l > 10.0 or max_s > 10.0:
                print(f"⚠️ [数值过大] {key}: LQ范围=[{min_l:.1f}, {max_l:.1f}] (疑似未归一化)")
                cnt_suspicious += 1
            elif max_l < -20.0 or max_s < -20.0:
                print(f"⚠️ [信号过弱] {key}: LQ最大值只有 {max_l:.1f} (疑似空数据)")
                cnt_suspicious += 1
                
        except Exception as e:
            print(f"❌ 读取错误 {key}: {e}")

    print("-" * 60)
    if cnt_suspicious == 0 and cnt_nan == 0:
        print("🎉 完美通过！所有数据数值范围正常 (约 -60 到 0 dB)，无错位。")
        print("👉 可以放心开始 Exp63 训练！")
    else:
        print(f"⚠️ 发现 {cnt_suspicious} 个数值可疑文件，请务必查看上方日志！")

if __name__ == "__main__":
    check_nifti_data_v3()

📂 数据集路径: /home/liujia/g_linux/Phantom_Carotid/
   LQ 文件数: 201
   SQ 文件数: 201
------------------------------------------------------------
✅ 智能配对成功: 201 对数据
------------------------------------------------------------
🚀 开始核心检查：数值范围 & NaN...


  0%|          | 0/201 [00:00<?, ?it/s]

100%|██████████| 201/201 [01:05<00:00,  3.08it/s]

------------------------------------------------------------
🎉 完美通过！所有数据数值范围正常 (约 -60 到 0 dB)，无错位。
👉 可以放心开始 Exp63 训练！
